# Análise de dados com Python - Portal da Transparência

In [24]:
caminho_dados = 'dados/2024_Viagem.csv'

In [25]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

## Lendo os dados

In [26]:
df_viagens = pd.read_csv(caminho_dados, sep=';', decimal=',', encoding='windows-1252')

## Criando nova coluna de Despesas 

In [27]:
df_viagens['Despesas'] = df_viagens['Valor diárias'] + df_viagens['Valor passagens'] + df_viagens['Valor outros gastos']

## Ajustando valores nulos na coluna de cargo

In [28]:
df_viagens['Cargo'] = df_viagens['Cargo'].fillna('Cargo não informado')

## Convertendo colunas de datas

In [29]:
df_viagens["Período - Data de início"] = pd.to_datetime(df_viagens["Período - Data de início"], format='%d/%m/%Y', errors='coerce')
df_viagens["Período - Data de fim"] = pd.to_datetime(df_viagens["Período - Data de fim"], format='%d/%m/%Y', errors='coerce')

## Criando novas colunas de datas

In [30]:
# Nome do mês da Viagem
df_viagens['Mês da Viagem'] = df_viagens['Período - Data de início'].dt.month_name()

# Dias de duração da viagem
df_viagens['Dias de duração'] = (df_viagens['Período - Data de fim'] - df_viagens['Período - Data de início']).dt.days